<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

**Цели:**
- Построить модель с предельно большим значением *F1*-меры. Нужно довести метрику до 0.59. Проверить *F1*-меру на тестовой выборке.

- Дополнительно измерить *AUC-ROC*, сравнить её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [1]:
# Импортирую необходимые библиотеки.  
!pip install imblearn
import pandas as pd
import numpy as np
from tqdm import trange
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    f1_score,
    roc_auc_score,
    precision_score,
    recall_score
    );

In [2]:
# Импорт данных  
data = pd.read_csv('/datasets/Churn.csv')

Изучу данные датасета

In [3]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


1) Видно, что в нескольких столбцах признаки категориальные, которые без дополнительного преобразования обработать не получится.

2) Количество данных: 10000, этого достаточно для проведения анализа.

3) В столбце "Tenure" (сколько лет клиент является клиентом банка) имеются пропуски. Заполню их медианным значением:  

In [5]:
data['Tenure'] = data['Tenure'].fillna(data['Tenure'].median())

In [1]:
# Избавлюсь от ненужных столбцов
data = data.drop(['RowNumber', 'CustomerId', 'Surname'] , axis=1)
data.info()

NameError: name 'data' is not defined

In [7]:
# Преобразую категориальные признаки в численные, техникой прямого кодирования.  
data_ohe = pd.get_dummies (data, drop_first= True )

In [8]:
data_ohe.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0


In [9]:
data_ohe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CreditScore        10000 non-null  int64  
 1   Age                10000 non-null  int64  
 2   Tenure             10000 non-null  float64
 3   Balance            10000 non-null  float64
 4   NumOfProducts      10000 non-null  int64  
 5   HasCrCard          10000 non-null  int64  
 6   IsActiveMember     10000 non-null  int64  
 7   EstimatedSalary    10000 non-null  float64
 8   Exited             10000 non-null  int64  
 9   Geography_Germany  10000 non-null  uint8  
 10  Geography_Spain    10000 non-null  uint8  
 11  Gender_Male        10000 non-null  uint8  
dtypes: float64(3), int64(6), uint8(3)
memory usage: 732.5 KB


In [10]:
data_ohe.describe()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
count,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,650.528800,38.921800,4.99790,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700,0.250900,0.247700,0.545700
std,96.653299,10.487806,2.76001,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769,0.433553,0.431698,0.497932
min,350.000000,18.000000,0.00000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000,0.000000,0.000000,0.000000
25%,584.000000,32.000000,3.00000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000,0.000000,0.000000,0.000000
50%,652.000000,37.000000,5.00000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000,0.000000,0.000000,1.000000
75%,718.000000,44.000000,7.00000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000,1.000000,0.000000,1.000000
max,850.000000,92.000000,10.00000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000,1.000000,1.000000,1.000000


In [11]:
# Выделю из данных "Признаки" и "Целевой признак".  
target = data_ohe['Exited']
features = data_ohe.drop(['Exited'] , axis=1)

In [12]:
# Разделю данные на "Обучающую выборку" 60% и "Валидационную выборку".  
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, random_state=12345, stratify=target) 

In [13]:
# Проверю качество разбиения.  
print("Признаки в Обучающей выборке:", features_train.shape, "Цель Обучающей выборки:", target_train.shape)
print("Признаки в Валидационной выборке:", features_valid.shape, "Цель Валидационной выборки:", target_valid.shape)

Признаки в Обучающей выборке: (6000, 11) Цель Обучающей выборки: (6000,)
Признаки в Валидационной выборке: (4000, 11) Цель Валидационной выборки: (4000,)


In [14]:
# Из "Валидационной выборки" выделю 20% на "Тестовую выборку".  
features_test, features_valid, target_test, target_valid = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345, stratify=target_valid)

In [15]:
# Проверю качество разбиения.  
print("Признаки в Валидационной выборке:", features_valid.shape, "Цель Валидационной выборки:", target_valid.shape)
print("Признаки в Тестовой выборке:", features_test.shape, "Цель Тестовой выборки:", target_test.shape)

Признаки в Валидационной выборке: (2000, 11) Цель Валидационной выборки: (2000,)
Признаки в Тестовой выборке: (2000, 11) Цель Тестовой выборки: (2000,)


<div class="alert alert-info">
В процессе предобработки данных, выполнил следующее:
    
- избавился от пропусков в столбце Tenure (сколько лет клиент работает с банком), заменив их медианным значением,
- преобразовал категориальные признаки (столбцы Surname (фамилия), Geography (страна проживания) и Gender (пол) в численные, техникой прямого кодирования,
- избавился от ненужных столбцов RowNumber (индекс строки в данных) и CustomerId (уникальный идентификатор клиента),
- из массива данных выделил признаки (2942столбца) и целевой признак (Exited - факт ухода клиента. Данные в нём категориальные, поэтому в дальнейшем буду использовать методы классификации),
- все данные поделил на Обучающую выборку (60%, 6000 строк), валидационную (20%, 2000 строк) и Тестовую (20%, 2000 строк)

Все данные предварительно подготовлены для проведения дальнейшего исследования, данных достаточно для обучения, проверки и тестирования моделей.
</div>

## Исследование задачи

In [16]:
# Проверю целевой признак на наличие дисбаланса
target_train.value_counts(normalize=True)

0    0.796333
1    0.203667
Name: Exited, dtype: float64

Имею довольно сильный дисбаланс (четырёхкратный), с которым в дальнейшем необходимо будет поработать.

Так как у наших признаков 'CreditScore', 'Age', 'Tenure', 'Balance', 'EstimatedSalary' разный масштаб, их необходимо стандартизовать (масштабировать) с помощью StandardScaler.

In [17]:
# Выберу столбцы, к которым применить скалирование, и применю его к обучающей, валидационной и тестовой выборкам
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'EstimatedSalary']
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])
features_train.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
2837,-1.040434,0.953312,0.360600,0.774657,1,0,1,-0.119110,1,0,0
9925,0.454006,-0.095244,-0.002786,1.910540,1,1,1,-0.258658,0,0,0
8746,0.103585,-0.476537,1.087371,0.481608,2,0,1,1.422836,0,0,1
660,-0.184996,0.190726,-0.002786,0.088439,1,1,1,-1.160427,1,0,0
3610,-0.720933,1.620574,-1.456328,0.879129,1,1,0,0.113236,0,0,0


<div class="alert alert-info">
Для бизнеса важно спрогнозировать, уйдёт ли клиент из банка в ближайшее время, или нет (т.е. спрогнозировать 1, положительный ответ (класс)). Для оценки качества прогноза положительного класса, использую метрики "Полнота" (Recall, описывает, как хорошо модель разобралась в особенностях этого положительного класса и распознала его) и "Точность" (Precision, выявляет, не переусердствует ли модель, присваивая положительные метки). Но используется F1-мера, а она агрегирует обе эти метрики.

Метрика качества AUC-ROC показывает, как сильно наша модель отличается от случайной (AUC-ROC случайной модели равна 0.5).

</div>

In [18]:
# Проверю модели на адекватность классификатором DummyClassifier.
model_dc = DummyClassifier(strategy='most_frequent', random_state=12345)
model_dc.fit(features_train, target_train)
f1_dc = model_dc.score(features_valid, target_valid)
print('f1 DummyClassifier:', f1_dc)

f1 DummyClassifier: 0.7965


In [19]:
# Построю модель Логистической регрессии, обучу её и предскажу значения на валидационной выборке, проверю F1 и AUC-ROC
model = LogisticRegression(random_state=12345, solver='liblinear', max_iter=1000)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print('F1:', f1_score(target_valid, predicted_valid))
probabilities_valid = model.predict_proba(features_valid)
print('AUC-ROC:', roc_auc_score(target_valid, probabilities_valid[:, 1]))

F1: 0.2932330827067669
AUC-ROC: 0.7504484453636997


F1-мера довольно низка (0,29). Метрика AUC-ROC получилась немного лучше 0,75 - выше, чем у случайной модели (0,5). F1-мера модели DummyClassifier (0.7965) выше, чем 0.29, значит данная модель неадекватна и неэффективна.

In [20]:
# Попробую применить модель "Дерево решений":  
best_f1 = 0
best_depth_tree = 0
best_model_tree = None

for depth in trange(1,16):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    
    predicted_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predicted_valid)
    
    if f1 > best_f1:
        best_f1 = f1
        best_depth_tree = depth
        best_model_tree = model
print("f1 наилучшей модели на валидационной выборке:", best_f1, "Глубина дерева:", best_depth_tree)
probabilities_valid = best_model_tree.predict_proba(features_valid)
print('AUC-ROC:', roc_auc_score(target_valid, probabilities_valid[:, 1]))

100%|██████████| 15/15 [00:00<00:00, 51.24it/s]

f1 наилучшей модели на валидационной выборке: 0.5533834586466165 Глубина дерева: 7
AUC-ROC: 0.8118303203048964


Показатели данной модели значительно выше. F1-мера уже поднялась до 0,55, но ниже пока требуемого значения в 0,59. Метрика AUC-ROC получилась уже 0,81. Оптимальная глубина дерева - 7.

F1-мера модели DummyClassifier (0.7965) выше, чем наша 0.56, данная модель неадекватна и неэффективна.

In [23]:
# Попробую применить модель "Случайный лес":  
best_f1_forest = 0
best_est = 0
best_depth_forest = 0
best_model_forest = None

for est in trange(27, 36):
    for depth in range(15, 25):
        model = RandomForestClassifier(n_estimators=est, max_depth=depth, random_state=12345)
        model.fit(features_train, target_train)
        predicted_valid = model.predict(features_valid)
        f1 = f1_score(target_valid, predicted_valid)
    
        if f1 > best_f1_forest:
            best_f1_forest = f1
            best_depth_forest = depth
            best_est = est
            best_model_forest = model

print("f1 лучшей модели:", best_f1_forest,"Наилучшая глубина дерева:", best_depth_forest,
      'Оптимальное количество деревьев:', best_est)
probabilities_valid = best_model_forest.predict_proba(features_valid)
print('AUC-ROC:', roc_auc_score(target_valid, probabilities_valid[:, 1]))

100%|██████████| 9/9 [00:20<00:00,  2.25s/it]

f1 лучшей модели: 0.5744360902255639 Наилучшая глубина дерева: 19 Оптимальное количество деревьев: 31
AUC-ROC: 0.8363016329118023


F1-мера у данной модели 0,5744 - выше, чем у Дерева решений. Метрика AUC-ROC на 0,02 поднялась: 0,83.

Оптимальная глубина дерева - 19, оптимальное количество деревьев: 31.

F1-мера модели DummyClassifier (0.7965) выше, чем 0.5744 данной модели, следовательно модель неадекватна и неэффективна.

## Борьба с дисбалансом

**Вначале сделаю апсемплинг данных**

In [25]:
oversample = SMOTE(random_state=12345)
features_train_up, target_train_up = oversample.fit_resample(features_train, target_train)

In [26]:
# Заново проверю дисбаланс
target_train_up.value_counts(normalize=True)

0    0.5
1    0.5
Name: Exited, dtype: float64

Дисбаланс устранён. Обучу модели на новых данных

In [27]:
# Логистическая регрессия
model_logist_up = LogisticRegression(random_state=12345, solver='liblinear', max_iter=1000)
model_logist_up.fit(features_train_up, target_train_up)
predicted_valid = model_logist_up.predict(features_valid)
print('F1:', f1_score(target_valid, predicted_valid))
probabilities_valid = model_logist_up.predict_proba(features_valid)
print('AUC-ROC:', roc_auc_score(target_valid, probabilities_valid[:, 1]))

F1: 0.46648793565683644
AUC-ROC: 0.7357388204845832


F1-мера хотя и выше, чем у модели с дисбалансом, но ещё не превысила необходимый нам порог в 0,59.

AUC-ROC наоборот немного ухудшилась (0,7357 против 0,7504 у модели с дисбалансом).

In [29]:
# модель "Дерево решений":  
best_f1 = 0
best_depth_tree = 0
best_model_tree_up = None

for depth in trange(1,11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train_up, target_train_up)
    
    predicted_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predicted_valid)
    
    if f1 > best_f1:
        best_f1 = f1
        best_depth_tree = depth
        best_model_tree_up = model
print("f1 наилучшей модели на валидационной выборке:", best_f1, "Глубина дерева:", best_depth_tree)
probabilities_valid = best_model_tree_up.predict_proba(features_valid)
print('AUC-ROC:', roc_auc_score(target_valid, probabilities_valid[:, 1]))

100%|██████████| 10/10 [00:00<00:00, 30.04it/s]

f1 наилучшей модели на валидационной выборке: 0.5681818181818182 Глубина дерева: 6
AUC-ROC: 0.8298336857658892


С данной моделью F1 и AUC-ROC получились чуть лучше, чем у модели с дисбалансом. Глубина дерева 6 

In [31]:
# Попробую применить модель "Случайный лес":  
best_f1_forest = 0
best_est = 0
best_depth_forest = 0
best_model_forest_up = None

for est in trange(8, 15):
    for depth in range(4, 10):
        model = RandomForestClassifier(n_estimators=est, max_depth=depth, random_state=12345)
        model.fit(features_train_up, target_train_up)
        predicted_valid = model.predict(features_valid)
        f1 = f1_score(target_valid, predicted_valid)
    
        if f1 > best_f1_forest:
            best_f1_forest = f1
            best_depth_forest = depth
            best_est = est
            best_model_forest_up = model

print("f1 лучшей модели:", best_f1_forest,"Наилучшая глубина дерева:", best_depth_forest,
      'Оптимальное количество деревьев:', best_est)
probabilities_valid = best_model_forest_up.predict_proba(features_valid)
print('AUC-ROC:', roc_auc_score(target_valid, probabilities_valid[:, 1]))

100%|██████████| 7/7 [00:03<00:00,  2.00it/s]

f1 лучшей модели: 0.6051282051282052 Наилучшая глубина дерева: 6 Оптимальное количество деревьев: 11
AUC-ROC: 0.8397565516209583


С данной моделью F1 стала чуть выше (0,605128), чем у модели с дисбалансом, и наконец превысила уровень 0,59. AUC-ROC также немного подросла.

**Сделаю under-sampling данных**

In [32]:
rus = RandomUnderSampler(random_state=12345)
features_train_down, target_train_down = rus.fit_resample(features_train, target_train)

In [33]:
# Заново проверю дисбаланс
target_train_down.value_counts(normalize=True)

0    0.5
1    0.5
Name: Exited, dtype: float64

In [34]:
# Логистическая регрессия
model_logist_down = LogisticRegression(random_state=12345, solver='liblinear', max_iter=1000)
model_logist_down.fit(features_train_down, target_train_down)
predicted_valid = model_logist_down.predict(features_valid)
print('F1:', f1_score(target_valid, predicted_valid))
probabilities_valid = model_logist_down.predict_proba(features_valid)
print('AUC-ROC:', roc_auc_score(target_valid, probabilities_valid[:, 1]))

F1: 0.47709593777009507
AUC-ROC: 0.7525213965891931


И F1 и AUC-ROC чуть выше, чем при апсемплинге, но другие модели (дерево решений и случайный лес) показали себя более эффективно.  

In [36]:
# модель "Дерево решений":  
best_f1 = 0
best_depth_tree = 0
best_model_tree_down = None

for depth in trange(1,11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train_down, target_train_down)
    
    predicted_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predicted_valid)
    
    if f1 > best_f1:
        best_f1 = f1
        best_depth_tree = depth
        best_model_tree_down = model
print("f1 наилучшей модели на валидационной выборке:", best_f1, "Глубина дерева:", best_depth_tree)
probabilities_valid = best_model_tree_down.predict_proba(features_valid)
print('AUC-ROC:', roc_auc_score(target_valid, probabilities_valid[:, 1]))

100%|██████████| 10/10 [00:00<00:00, 97.72it/s]

f1 наилучшей модели на валидационной выборке: 0.5637583892617449 Глубина дерева: 6
AUC-ROC: 0.8217562709088133


F1 и AUC-ROC получились практически такие же, но чуть хуже, чем при апсемплинге. Глубина дерева 6 

In [38]:
# Попробую применить модель "Случайный лес":  
best_f1_forest = 0
best_est = 0
best_depth_forest = 0
best_model_forest_down = None

for est in trange(24, 31):
    for depth in range(5, 12):
        model = RandomForestClassifier(n_estimators=est, max_depth=depth, random_state=12345)
        model.fit(features_train_down, target_train_down)
        predicted_valid = model.predict(features_valid)
        f1 = f1_score(target_valid, predicted_valid)
    
        if f1 > best_f1_forest:
            best_f1_forest = f1
            best_depth_forest = depth
            best_est = est
            best_model_forest_down = model

print("f1 лучшей модели:", best_f1_forest,"Наилучшая глубина дерева:", best_depth_forest,
      'Оптимальное количество деревьев:', best_est)
probabilities_valid = best_model_forest_down.predict_proba(features_valid)
print('AUC-ROC:', roc_auc_score(target_valid, probabilities_valid[:, 1]))

100%|██████████| 7/7 [00:04<00:00,  1.58it/s]

f1 лучшей модели: 0.5952153110047846 Наилучшая глубина дерева: 8 Оптимальное количество деревьев: 27
AUC-ROC: 0.8454571674910657


С данной моделью F1 чуть ниже (0,5952), чем у модели с апсемплингом (0,60513), и также превысила уровень 0,59. AUC-ROC по сравнению с апсемплингом немного подросла. Глубина дерева 8, количество деревьев 27.

**Сделаю проверку моделей встроенной балансировкой классов, без устранения дисбаланса**

In [39]:
# Логистическая регрессия
model_logist_weigh = LogisticRegression(random_state=12345, class_weight='balanced', solver='liblinear', max_iter=1000)
model_logist_weigh.fit(features_train, target_train)
predicted_valid = model_logist_weigh.predict(features_valid)
print('F1:', f1_score(target_valid, predicted_valid))
probabilities_valid = model_logist_weigh.predict_proba(features_valid)
print('AUC-ROC:', roc_auc_score(target_valid, probabilities_valid[:, 1]))

F1: 0.47478260869565214
AUC-ROC: 0.7541779067202796


F1 немного проигрывает регрессии с даунсемплингом, но лучше двух остальных (с дисбалансом и с апсемплингом) (0,4748), до уровня 0,59 не дотягивает.
AUC-ROC здесь получили немного лучше даунсемплинга.

In [41]:
# модель "Дерево решений":  
best_f1 = 0
best_depth_tree = 0
best_model_tree_weigh = None

for depth in trange(4,11):
    model = DecisionTreeClassifier(random_state=12345, class_weight='balanced', max_depth=depth)
    model.fit(features_train, target_train)
    
    predicted_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predicted_valid)
    
    if f1 > best_f1:
        best_f1 = f1
        best_depth_tree = depth
        best_model_tree_weigh = model
print("f1 наилучшей модели на валидационной выборке:", best_f1, "Глубина дерева:", best_depth_tree)
probabilities_valid = best_model_tree_weigh.predict_proba(features_valid)
print('AUC-ROC:', roc_auc_score(target_valid, probabilities_valid[:, 1]))

100%|██████████| 7/7 [00:00<00:00, 47.49it/s]

f1 наилучшей модели на валидационной выборке: 0.5456204379562044 Глубина дерева: 7
AUC-ROC: 0.7920347157635294


F1 получил даже хуже, чем в модели без какого-либо устранения дисбаланса (0,55338).

AUC-ROC имеем также наихудший результат среди всех моделей "Дерево решений".

Глубина дерева 7.

In [43]:
# Попробую применить модель "Случайный лес":  
best_f1_forest = 0
best_est = 0
best_depth_forest = 0
best_model_forest_weigh = None

for est in trange(10, 17):
    for depth in range(6, 13):
        model = RandomForestClassifier(n_estimators=est, max_depth=depth, class_weight='balanced', random_state=12345)
        model.fit(features_train, target_train)
        predicted_valid = model.predict(features_valid)
        f1 = f1_score(target_valid, predicted_valid)
    
        if f1 > best_f1_forest:
            best_f1_forest = f1
            best_depth_forest = depth
            best_est = est
            best_model_forest_weigh = model

print("f1 лучшей модели:", best_f1_forest,"Наилучшая глубина дерева:", best_depth_forest,
      'Оптимальное количество деревьев:', best_est)
probabilities_valid = best_model_forest_weigh.predict_proba(features_valid)
print('AUC-ROC:', roc_auc_score(target_valid, probabilities_valid[:, 1]))

100%|██████████| 7/7 [00:03<00:00,  1.78it/s]

f1 лучшей модели: 0.6139534883720931 Наилучшая глубина дерева: 9 Оптимальное количество деревьев: 13
AUC-ROC: 0.8487940945568064


F1 (0,6139) и AUC-ROC (0,8488) данной модели имеют самые высокие значения среди всех рассмотренных. Условие по превышению метрикой F1 значения 0,59 выполнено.

**Итак, наилучшие результаты были достигнуты при использовании встроенной балансировки классов, с моделью "Случайный лес", при использовании гиперпараметров: оптимальная глубина дерева - 9, оптимальное количество деревьев: 13.**

In [44]:
# Дообучу выбранную модель RF, на обучающей + валидационной выборке:
best_model_forest_weigh.fit(pd.concat([features_train_up, features_valid]), pd.concat([target_train_up, target_valid]));

In [45]:
probabilities_valid = best_model_forest_weigh.predict_proba(features_valid)
print("f1 дообученной модели на валидационной выборке:", f1_score(target_valid,
                                                             best_model_forest_weigh.predict(features_valid)))
print('AUC-ROC дообученной модели на валидационной выборке:', roc_auc_score(target_valid, probabilities_valid[:, 1]))

f1 дообученной модели на валидационной выборке: 0.6711259754738015
AUC-ROC дообученной модели на валидационной выборке: 0.9086883493663156


И f1, и AUC-ROC теперь значительно увеличились.

In [46]:
# Рассчитаю метрику RECALL (Полнота)
print('Recall:', recall_score(target_valid, best_model_forest_weigh.predict(features_valid), pos_label=1))

Recall: 0.7395577395577395


**Вывод:**

F1=0.67, это максимальное значение, которого удалось добиться. Цель довести метрику до 0,59 достигнута.

Recall=0.74, а данная метрика показывает долю истинно-положительных ответов (хорошее значение).

AUC-ROC= 0.91 ~ 1, а это площадь под ROC-кривой, и если она равна 1, имеем "идеальную" модель. AUC-ROC случайной модели = 0,5

## Тестирование модели

In [47]:
probabiliti = best_model_forest_weigh.predict(features_test)
print('f1 на тестовой выборке с использованием встроенной балансировки классов:', f1_score(target_test, probabiliti))
probabilities_test = best_model_forest_weigh.predict_proba(features_test)
print('AUC-ROC на тестовой выборке с использованием встроенной балансировки классов:',
      roc_auc_score(target_test, probabilities_test[:, 1]))

f1 на тестовой выборке с использованием встроенной балансировки классов: 0.6277056277056278
AUC-ROC на тестовой выборке с использованием встроенной балансировки классов: 0.8587968950142871


f1 и AUC-ROC на тестовой выборке получились чуть ниже, чем на валидационной + обучающей выборках, но довольно высоки, f1-мера выше требуемого в задании порога в 0,59.

In [48]:
print('Количество клиентов, покинувших банк:', sum(target))

Количество клиентов, покинувших банк: 2037


In [49]:
print('Прогнозируемое количество клиентов, которые ещё могут уйти из банка в ближайшее время:',
      sum(best_model_forest_weigh.predict(features)) - sum(target))

Прогнозируемое количество клиентов, которые ещё могут уйти из банка в ближайшее время: 1012


**ИТОГОВЫЕ ВЫВОДЫ по прогнозированию оттока клиентов**
Итак, в результате проведённой работы выбраны следующие параметры:
- так как в данных имеем сильный четырёхкратный дисбаланс классов, к ним поочерёдно применял апсемплинг методом SMOTE, даунсемплинг методом RANDOMUNDERSAMPLER, а также использовал встроенные средства моделей для балансировки классов.
- наилучшие результаты показала модель "Случайный лес", при использовании следующих гиперпараметров: глубина дерева - 79 (по умолчанию нет, ветвление идёт только вширь по количеству деревьев), количество деревьев: 31 (по умолчанию 100), с включенным встроенным средством балансировки.
- благодаря таким настройкам данная модель показывает высокие основные для нашей модели метрики "Полнота" и "Точность", характеризующие качество предсказания положительного класса ("клиент покинул банк")
- после обучения, валидации и тестирования модели, на имеющихся данных (10 000 клиентов, из которых 2 037 уже покинуло банк), модель спрогнозировала уход ещё 1012 клиентов.